In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import _tree
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pylab as plt

from itertools import combinations
#del list

import math

In [2]:
Indt_core_train = pd.read_csv('Indt_CORE_train_2019-04-09.csv')
Indt_core_test = pd.read_csv('Indt_CORE_test_2019-04-09.csv')

In [3]:
Indt_core_test.shape

(2568, 1134)

In [4]:
# 魔蝎蜂巢合并 2018.08.20 仅训练集
row = Indt_core_train.shape[0]
mean_var44_train = Indt_core_train[Indt_core_train['var44']>0]['var44'].mean()
mean_var46_train = Indt_core_train[Indt_core_train['var46']>0]['var46'].mean()
mean_var47_train = Indt_core_train[Indt_core_train['var47']>0]['var47'].mean()
for i in range(row):
    if (Indt_core_train.ix[i,'var44'] == -3) or (Indt_core_train.ix[i,'var44'] == -2): #魔蝎_入网时长
        Indt_core_train.ix[i,'var44'] = mean_var44_train #Indt_core_train.ix[i,'var51']
    if (Indt_core_train.ix[i,'var46'] == -3) or (Indt_core_train.ix[i,'var46'] == -2): #魔蝎_夜间活动比例
        Indt_core_train.ix[i,'var46'] = mean_var46_train #Indt_core_train.ix[i,'var50']
    if (Indt_core_train.ix[i,'var47'] == -3) or (Indt_core_train.ix[i,'var47'] == -2): #魔蝎_电话互通数量
        Indt_core_train.ix[i,'var47'] = mean_var47_train #Indt_core_train.ix[i,'var48']   

In [5]:
mean_var8_train = Indt_core_train[Indt_core_train['var8']<999]['var8'].mean()
mean_var8_test = Indt_core_test[Indt_core_test['var8']<999]['var8'].mean()        
Indt_core_train['var8'] = Indt_core_train['var8'].map(lambda x: mean_var8_train if x==9999 else x) #算话_30天内单位信息被使用人数
Indt_core_test['var8'] = Indt_core_test['var8'].map(lambda x: mean_var8_test if x==9999 else x) #算话_30天内单位信息被使用人数

mean_var7_train = Indt_core_train[Indt_core_train['var7']<999]['var7'].mean()
mean_var7_test = Indt_core_test[Indt_core_test['var7']<999]['var7'].mean()        
Indt_core_train['var7'] = Indt_core_train['var7'].map(lambda x: mean_var7_train if x==9999 else x) #算话_30天内单位信息被使用人数
Indt_core_test['var7'] = Indt_core_test['var7'].map(lambda x: mean_var7_test if x==9999 else x) #算话_30天内单位信息被使用人数

# var27-var42 删除;  var48-var54 删除;  var1-var13 删除
col = []
for i in range(1,55):
    if i<=13 or (i>=27 and i<43) or (i>=48 and i<55):
        col.append("var" + str(i))
        col.append("var" + str(i) + "_cnt")
Indt_core_train.drop(columns = col,inplace=True)
Indt_core_test.drop(columns = col,inplace=True)

# 删除缺失值>10%的列
cnt = []
for i in Indt_core_train.columns:
    if "_cnt" in i:
        cnt.append(i)
cnt.remove('score_v1_cnt')
name = Indt_core_train[cnt].columns
missing = []
for i in name:
    if Indt_core_train.ix[:,i].mean() < 0.9:
        missing.append(i[0:5])
Indt_core_train.drop(columns = missing,inplace=True)
Indt_core_test.drop(columns = missing,inplace=True)

In [6]:
def var(df):
    name = df.columns
    
    for i in name:
        if "_cnt" in i:
            df = df.drop(columns = [i])

    df = df.drop(columns = ['LOAN_NO', 'CERT_NO', 'PHONE_NO', 'CUST_NAME', 'REG_DATE',
       'CHANNEL_NAME', 'APPLY_DATE', 'PROD_NO', 'PAY_AMT', 'LOAN_DATE','LOAN_AMT', 
       'ACT_LOAN_AMT', 'DUE_DATE', 'ACT_REPAY_DATE', 'REPAY_STAT','OVDU_DAYS',
       'LOAN_NO.1', 'score_v1','layer','cert_no','nDay_Next_Odue','cust_name.x','apply_date.x','cust_name.y','apply_date.y',
       'latest_loan_date','latest_query_date'])
    return df

Indt_core_train = var(Indt_core_train)
Indt_core_test = var(Indt_core_test)

In [7]:
def value_count(df):
    name=df.columns
    lenth=[]
    names=[]
    for i in name:
        lenth.append(len(df[i].value_counts()))
        names.append(i)
    lenth=pd.DataFrame(lenth)
    names=pd.DataFrame(names)
    valuecounts=pd.concat([names,lenth],axis=1)
    valuecounts.columns=["name","vary_num"]
    return valuecounts

# 删除只有一值的数据
df = value_count(Indt_core_train)
list = df[df['vary_num'] == 1]['name'].tolist()
Indt_core_train.drop(columns = list,inplace=True)
Indt_core_test.drop(columns = list,inplace=True)

In [8]:
Indt_core_train['jyzx_min_Inquiry_date_gap'] = Indt_core_train['jyzx_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['jyzx_min_Inquiry_date_gap'] = Indt_core_test['jyzx_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_train['jyzx_max_Inquiry_date_gap'] = Indt_core_train['jyzx_max_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['jyzx_max_Inquiry_date_gap'] = Indt_core_test['jyzx_max_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_train['af_min_Inquiry_date_gap'] = Indt_core_train['af_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['af_min_Inquiry_date_gap'] = Indt_core_test['af_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)

In [9]:
# 客户婚姻状况分类
# 84000001 未婚;  84000002 已婚有子女;  84000003 已婚无子女;  84000004 离异;  84000005 丧偶
Indt_core_train['ind_married'] = Indt_core_train['var61'].map(lambda x: 1 if (x==84000001 or x==84000004) else 0)
Indt_core_train['ind_divorced'] = Indt_core_train['var61'].map(lambda x: 0 if (x==84000004) else 1)
Indt_core_train['ind_parent'] = Indt_core_train['var61'].map(lambda x: 0 if (x==84000002) else 1)
Indt_core_test['ind_married'] = Indt_core_test['var61'].map(lambda x: 1 if (x==84000001 or x==84000004) else 0)
Indt_core_test['ind_divorced'] = Indt_core_test['var61'].map(lambda x: 0 if (x==84000004) else 1)
Indt_core_test['ind_parent'] = Indt_core_test['var61'].map(lambda x: 0 if (x==84000002) else 1)

In [10]:
train = Indt_core_train
test = Indt_core_test
X_train = Indt_core_train.drop("target",axis=1)
y_train = Indt_core_train["target"]
X_test = Indt_core_test.drop("target",axis=1)
y_test = Indt_core_test["target"]

In [11]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.99 * (1 - .99))) 

s = sel.fit_transform(X_train)

indices = sel.get_support(False)
feat_labels = X_train.columns
features = feat_labels[indices]
X_train = X_train.loc[:,features]
X_test = X_test.loc[:,features]

In [12]:
from sklearn.feature_selection import SelectKBest,chi2,SelectFpr
from sklearn.feature_selection import GenericUnivariateSelect

sp = SelectFpr(chi2, alpha=0.05)
s = sp.fit_transform(X_train, y_train)

indices = sp.get_support(False)
feat_labels = X_train.columns
features = feat_labels[indices]
X_train = X_train.loc[:,features]
X_test = X_test.loc[:,features]

In [13]:
X_test.shape

(2568, 535)

In [14]:
# 计算斯皮尔曼相关系数（矩阵）
X_train_corr = X_train.corr(method='spearman')

# 将对角线变为0
mask = np.ones(X_train_corr.columns.size) - np.eye(X_train_corr.columns.size)
X_train_corr = mask * X_train_corr

drops = []
for col in X_train_corr.columns.values:
    # if we've already determined to drop the current variable, continue
    if np.in1d([col],drops):
        continue

    # 找出高相关的变量
    corr = X_train_corr[abs(X_train_corr[col]) > 0.98].index
    drops = np.union1d(drops, corr)

print("nDropping", drops.shape[0], "highly correlated features...n", drops)

nDropping 259 highly correlated features...n ['af_max_Inquiry_Checking_all' 'af_max_Repay_M1plus_180d'
 'af_max_Repay_M1plus_30d' 'af_max_Repay_M1plus_360d'
 'af_max_Repay_M1plus_90d' 'af_max_Repay_M1plus_all'
 'af_max_Repay_Odue_30d' 'af_max_Repay_Odue_all' 'af_max_Type_Credit_180d'
 'af_max_Type_Credit_30d' 'af_max_Type_Credit_360d'
 'af_max_Type_Credit_90d' 'af_max_Type_Credit_all' 'af_min_Odue_date_gap'
 'af_n_Repay_Odue_all' 'af_n_Type_Credit_180d' 'af_n_Type_Credit_30d'
 'af_n_Type_Credit_360d' 'af_n_Type_Credit_90d' 'af_n_Type_Credit_all'
 'af_sum_Inquiry_Aborted_180d' 'af_sum_Inquiry_Aborted_30d'
 'af_sum_Inquiry_Aborted_360d' 'af_sum_Inquiry_Aborted_90d'
 'af_sum_Inquiry_Aborted_all' 'af_sum_Inquiry_Approved_90d'
 'af_sum_Inquiry_Checking_180d' 'af_sum_Inquiry_Checking_30d'
 'af_sum_Inquiry_Checking_360d' 'af_sum_Inquiry_Checking_all'
 'af_sum_Repay_Clear_180d' 'af_sum_Repay_Clear_30d'
 'af_sum_Repay_Clear_360d' 'af_sum_Repay_Clear_90d' 'af_sum_Repay_M1_180d'
 'af_sum_Repay_M1

In [15]:
X_train = X_train.drop(columns = drops)
X_test = X_test.drop(columns = drops)

In [16]:
X_train.shape

(5716, 276)

# lightgbm

In [19]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = {
              #'max_depth': [5, 8, 9, 10, 11],
              #'learning_rate': [0.01, 0.05],
              #'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              #'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              #'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]
}

gbm = lgb.LGBMClassifier(boosting_type='gbdt',
                         objective = 'binary',
                         metric = 'auc',
                         verbose = 0,
                         max_depth = 10,
                         learning_rate = 0.01,
                         feature_fraction = 0.9,
                         bagging_fraction = 0.7,
                         bagging_freq = 8)
                         #num_leaves = 35,
                         #lambda_l1= 0.6,
                         #lambda_l2= 0)

# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='accuracy', cv=3)
gsearch.fit(X_train, y_train)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Best score: 0.830
Best parameters set:
	cat_smooth: 1
	lambda_l1: 0
	lambda_l2: 0


In [66]:
model = lgb.LGBMClassifier(max_depth=10, learning_rate=0.01, feature_fraction=0.9 , bagging_fraction=0.7 ,bagging_freq=8 ,objective='binary')
model.fit(X_train, y_train)

LGBMClassifier(bagging_fraction=0.7, bagging_freq=8, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        importance_type='split', learning_rate=0.01, max_depth=10,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [67]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1] #返回数组值从小到大的索引值  
                                        # np.argsort(-x)

feat_labels = X_train.columns
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) af_n_Query_NoLicConFin_all     104.000000
 2) apply_admittance_score         86.000000
 3) latest_1mth_query_num          82.000000
 4) past_with_success_num          78.000000
 5) af_n_Query_LoanCheck_all       78.000000
 6) var47                          72.000000
 7) af_n_Inquiry_Rejected_30d      67.000000
 8) var55                          67.000000
 9) var44                          62.000000
10) cridiet_loan_duration          59.000000
11) latest_6mth_query_num          52.000000
12) latest_1mth_with_failed_num    51.000000
13) af_n_Inquiry_Pass_rate_30d     47.000000
14) jyzx_max_Inquiry_date_gap      44.000000
15) loan_behavior_score            42.000000
16) af_max_Inquiry_date_gap        41.000000
17) jyzx_min_Inquiry_date_gap      39.000000
18) var16                          38.000000
19) af_sum_Inquiry_Approved_all    37.000000
20) af_sum_Inquiry_90d             36.000000
21) jyzx_mean_Period_all           35.000000
22) apply_admittance_confidence_level 35.000000
23) qu

In [71]:
indices_ = indices[0:273]
X_train_model = X_train.iloc[:,indices_]
X_test_model = X_test.iloc[:,indices_]

In [70]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.27771523788302993
KS-test: 0.25151422609049723


In [64]:
X_train_model.columns

Index(['af_n_Inquiry_Rejected_30d', 'af_n_Query_NoLicConFin_all',
       'af_n_Repay_Clear_all', 'af_n_Inquiry_Pass_rate_90d',
       'jyzx_max_Repay_Unknown_360d', 'loan_repay', 'af_max_Repay_Clear_30d',
       'af_n_Inquiry_Approved_all', 'jyzx_max_Repay_Paid_360d',
       'af_n_Inquiry_Pass_rate_180d',
       ...
       'jyzx_sum_Repay_Clear_all', 'jyzx_sum_Inquiry_Checking_360d',
       'jyzx_n_Repay_Unknown_360d', 'af_n_Query_P2P_all',
       'jyzx_sum_Inquiry_Approved_90d', 'af_n_Repay_Clear_30d',
       'jyzx_sum_Inquiry_Approved_180d', 'af_n_Repay_Clear_360d',
       'af_sum_Inquiry_360d', 'jyzx_sum_Inquiry_180d'],
      dtype='object', length=273)

In [42]:
del list

In [78]:
cols = ['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d', 'var55',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration', 'loan_rank',
       'latest_6mth_query_num', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d']

for var in combinations(cols, 18):
    li = list(var)
    X_train_model_3 = X_train_model[li]
    X_test_model_3 = X_test_model[li]
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train_model_3, y_train)
    
    predict_train = logistic_model.predict_proba(X_train_model_3)
    fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
    KS_train = max(tpr-fpr)

    if KS_train>0.29:
        predict_test = logistic_model.predict_proba(X_test_model_3)
        fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
        KS_test = max(tpr-fpr)

        if KS_test>0.29:
            print(X_train_model_3.columns,"KS_train:",KS_train,"KS_test",KS_test)
    
    del li

Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d', 'var55',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration',
       'latest_6mth_query_num', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d'],
      dtype='object') KS_train: 0.2973400630996321 KS_test 0.29440881559525633
Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d', 'var55',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'loan_rank', 'latest_6mth_query_num', 'var16',
       'af_n_Inquiry_30d', 'af_max_Repay_Odue_90d'],
      dtype='object') KS_train: 0.29503674353368703 KS_test 0.2936

Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration', 'loan_rank',
       'latest_6mth_query_num', 'var16', 'af_max_Repay_Odue_90d',
       'jyzx_n_Inquiry_180d'],
      dtype='object') KS_train: 0.29408992612542206 KS_test 0.30239985748460324
Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration',
       'latest_6mth_query_num', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'],
      dtype='object') KS_

In [ ]:
Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'af_n_Inquiry_Rejected_30d', 'var55',
       'jyzx_min_Inquiry_date_gap', 'apply_admittance_score',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration', 'loan_rank',
       'latest_6mth_query_num', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'],
      dtype='object') KS_train: 0.298672022923401 KS_test 0.30499567363974145
Index(['af_n_Query_LoanCheck_all', 'af_n_Query_NoLicConFin_all', 'var44',
       'past_with_success_num', 'var47', 'latest_1mth_query_num',
       'af_n_Inquiry_Rejected_30d', 'var55', 'latest_1mth_with_failed_num',
       'af_n_Inquiry_Pass_rate_30d', 'var15', 'af_n_Repay_Clear_all',
       'cridiet_loan_duration', 'loan_rank', 'latest_6mth_query_num',
       'af_sum_Inquiry_Pass_rate_360d', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'],
      dtype='object') KS_train: 0.2978920503290072 KS_test 0.30133099200895813
Index(['af_n_Query_LoanCheck_all', 'var44', 'past_with_success_num', 'var47',
       'latest_1mth_query_num', 'var55', 'jyzx_min_Inquiry_date_gap',
       'latest_1mth_with_failed_num', 'af_n_Inquiry_Pass_rate_30d', 'var15',
       'af_n_Repay_Clear_all', 'cridiet_loan_duration', 'loan_rank',
       'latest_6mth_query_num', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'],
      dtype='object') KS_train: 0.2972500745419572 KS_test 0.30829134218964727

# xgboost

In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

#分类器使用 xgboost
clf1 = xgb.XGBClassifier(objective='binary:logistic')
 
#设定网格搜索的xgboost参数搜索范围，值搜索XGBoost的主要6个参数
param_dist = {        
        'max_depth':range(1,3,1), #overfitting
        'learning_rate':[0.01, 0.1, 0.2], #overfitting
        'min_child_weight':range(5,7,1) #overfitting
    
        #'subsample':np.linspace(0.7,0.9,20), #speed
        #'colsample_bytree':np.linspace(0.5,0.98,10), #speed
        #'n_estimators':range(80,200,4), #speed
        }
 

#param_dist字典类型，放入参数搜索范围
#n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
grid = GridSearchCV(clf1, param_dist, cv=3, scoring='roc_auc')
 
#在训练集上训练
grid.fit(X_train, y_train)
#返回最优的训练器
best_estimator = grid.best_estimator_
print(best_estimator)
#输出最优训练器的精度
print(grid.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=6, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
0.669775154043883


In [53]:
model = xgb.XGBClassifier(max_depth=2, learning_rate=0.1, min_child_weight=6 ,silent=True, subsample=1, objective='binary:logistic')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=6, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [54]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1] #返回数组值从小到大的索引值  
                                        # np.argsort(-x)

feat_labels = X_train.columns
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) af_n_Inquiry_Rejected_30d      0.049512
 2) af_n_Query_NoLicConFin_all     0.030876
 3) af_n_Repay_Clear_all           0.027361
 4) af_n_Inquiry_Pass_rate_90d     0.026837
 5) jyzx_max_Repay_Unknown_360d    0.023384
 6) loan_repay                     0.021676
 7) af_max_Repay_Clear_30d         0.019994
 8) af_n_Inquiry_Approved_all      0.019190
 9) jyzx_max_Repay_Paid_360d       0.018867
10) af_n_Inquiry_Pass_rate_180d    0.018733
11) past_with_success_num          0.018224
12) loan_rank                      0.017673
13) af_n_Inquiry_Pass_rate_30d     0.017660
14) af_n_Repay_Clear_90d           0.017238
15) loan_orgnization_num           0.017210
16) jyzx_n_Inquiry_all             0.017198
17) latest_6mth_query_num          0.016941
18) af_max_Repay_Clear_360d        0.016806
19) af_n_Query_LoanCheck_all       0.016800
20) af_max_Repay_Odue_90d          0.015889
21) af_n_Repay_Clear_180d          0.015326
22) query_net_loan_orgnization_num 0.015124
23) af_sum_Inquiry_30d          

In [302]:
indices_ = indices[0:13]
X_train_model = X_train.iloc[:,indices_]
X_test_model = X_test.iloc[:,indices_]

In [303]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.39779411764705885
KS-test: 0.33076923076923076


In [304]:
X_train_model.columns

Index(['loan_repay', 'var15', 'past_with_failed_num', 'loan_rank',
       'latest_6mth_query_num', 'var17', 'cridiet_loan_duration',
       'latest_1mth_with_success_num', 'past_with_success_num', 'loan_ovdu',
       'af_max_Inquiry_Aborted_30d', 'jyzx_max_Repay_Normal_180d',
       'jyzx_max_Repay_Clear_180d'],
      dtype='object')

In [305]:
del list

NameError: name 'list' is not defined

In [307]:
cols = ['loan_repay', 'var15', 'past_with_failed_num', 'loan_rank',
       'latest_6mth_query_num', 'var17', 'cridiet_loan_duration',
       'latest_1mth_with_success_num', 'past_with_success_num', 'loan_ovdu',
       'af_max_Inquiry_Aborted_30d', 'jyzx_max_Repay_Normal_180d']

for var in combinations(cols, 3):
    li = list(var)
    X_train_model_3 = X_train_model[li]
    X_test_model_3 = X_test_model[li]
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train_model_3, y_train)
    
    predict_train = logistic_model.predict_proba(X_train_model_3)
    fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
    KS_train = max(tpr-fpr)

    if KS_train>0.35:
        predict_test = logistic_model.predict_proba(X_test_model_3)
        fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
        KS_test = max(tpr-fpr)

        if KS_test>0.45:
            print(X_train_model_3.columns,"KS_train:",KS_train,"KS_test",KS_test)
    
    del li

# random forest

In [231]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

forest = RandomForestClassifier()

param_dist = {
        'max_depth':range(2,10,1), #overfitting
        'min_samples_split':range(110, 201, 10),
        'max_features':range(5,11,1),
        'min_samples_leaf':range(10, 80, 10) #overfitting 
        }

#n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
grid = GridSearchCV(forest, param_dist, cv=3, scoring='roc_auc')
grid.fit(X_train, y_train)

best_estimator = grid.best_estimator_ #返回最优的训练器
print(best_estimator)
print(grid.best_score_)
display(pd.DataFrame(grid.cv_results_).T)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=110,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.5


,0,1,2,3,4,5,6,7,8,9,...,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359
mean_fit_time,0.0206807,0.0293539,0.0120087,0.013677,0.0146761,0.012677,0.0131005,0.0156269,0.0156252,0.0266808,...,0.010416,0.0104183,0.0156252,0.00520738,0.0104169,0.0159616,0.0233503,0.0178786,0.0105462,0.0156263
std_fit_time,0.00286885,0.0110932,0.000817022,0.000470083,0.00125044,0.000472778,0.00817549,1.59343e-06,3.89336e-07,0.00612812,...,0.00736525,0.00736682,9.19965e-07,0.00736435,0.00736587,0.0126257,0.00170454,0.00437884,0.00745892,2.95442e-06
mean_score_time,0.00467014,0.00500433,0.00466959,0.00300097,0.00366886,0.00366871,0.00200152,0.00520754,0,0.0106839,...,0,0.00521024,0,0.00520841,0.00520873,0.0092109,0.00733995,0.00400289,0.0142718,0
std_score_time,0.000944089,0.000818092,0.00124749,1.52041e-06,0.000943528,0.000471483,0.00141529,0.00736457,0,0.000478496,...,0,0.00736839,0,0.00736581,0.00736626,0.00515498,0.00236226,0.00294603,0.00870094,0
param_max_depth,2,2,2,2,2,2,2,2,2,2,...,9,9,9,9,9,9,9,9,9,9
param_max_features,5,5,5,5,5,5,5,5,5,5,...,10,10,10,10,10,10,10,10,10,10
param_min_samples_leaf,10,10,10,10,10,10,10,10,10,10,...,70,70,70,70,70,70,70,70,70,70
param_min_samples_split,110,120,130,140,150,160,170,180,190,200,...,110,120,130,140,150,160,170,180,190,200
params,"{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...","{'max_depth': 2, 'max_features': 5, 'min_sampl...",...,"{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp...","{'max_depth': 9, 'max_features': 10, 'min_samp..."
split0_test_score,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [232]:
forest = RandomForestClassifier(max_features=5, max_depth=2, min_samples_leaf=10, min_samples_split=110, n_estimators=10)
forest.fit(X_train, y_train)

importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = X_train.columns
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) loan_repay                     0.300000
 2) var15                          0.300000
 3) past_with_failed_num           0.100000
 4) loan_rank                      0.100000
 5) latest_6mth_query_num          0.100000
 6) var17                          0.100000
 7) cridiet_loan_duration          0.000000
 8) latest_1mth_with_success_num   0.000000
 9) past_with_success_num          0.000000
10) loan_ovdu                      0.000000
11) af_max_Inquiry_Aborted_30d     0.000000
12) jyzx_max_Repay_Normal_180d     0.000000
13) jyzx_max_Repay_Clear_180d      0.000000
14) jyzx_max_Inquiry_30d           0.000000
15) jyzx_max_Inquiry_180d          0.000000
16) var44                          0.000000


In [91]:
indices_ = indices[0:32]
X_train_model = X_train.iloc[:,indices_]
X_test_model = X_test.iloc[:,indices_]

In [92]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.262819662343693
KS-test: 0.2012753919654231


In [96]:
X_train_model.columns

Index(['af_n_Inquiry_Rejected_180d', 'var44', 'af_n_Query_LoanCheck_all',
       'consumer_orgnization_num', 'apply_admittance_score',
       'jyzx_max_Inquiry_30d', 'jyzx_n_Inquiry_Rejected_90d',
       'jyzx_n_Inquiry_all', 'af_n_Repay_Clear_90d',
       'af_sum_Inquiry_Approved_all', 'latest_6mth_loan_num',
       'jyzx_n_Repay_Unclear_30d', 'af_n_Inquiry_360d', 'af_sum_Inquiry_360d',
       'query_net_loan_orgnization_num', 'loan_rank',
       'jyzx_n_Inquiry_Pass_360d', 'jyzx_max_Inquiry_360d',
       'latest_1mth_query_num', 'net_loan_orgnization_num',
       'cridiet_loan_duration', 'af_sum_Inquiry_Rejected_all',
       'jyzx_min_Inquiry_date_gap', 'latest_1mth_with_success_num',
       'jyzx_mean_Period_all', 'jyzx_max_Repay_Clear_all',
       'af_n_Inquiry_Rejected_all', 'jyzx_max_Inquiry_all',
       'af_sum_Inquiry_90d', 'jyzx_n_Inquiry_360d', 'latest_3mth_loan_num',
       'var47'],
      dtype='object')

In [109]:
cols = ['af_n_Inquiry_Rejected_180d', 'var44', 'af_n_Query_LoanCheck_all',
       'consumer_orgnization_num', 'apply_admittance_score',
       'jyzx_max_Inquiry_30d', 'jyzx_n_Inquiry_Rejected_90d',
       'af_n_Repay_Clear_90d', 'latest_6mth_loan_num',
       'jyzx_n_Repay_Unclear_30d', 'af_n_Inquiry_360d',
       'jyzx_n_Inquiry_Pass_360d', 'latest_1mth_query_num',
       'net_loan_orgnization_num', 'jyzx_mean_Period_all', 'var47']

for var in combinations(cols, 14):
    li = list(var)
    X_train_model_3 = X_train_model[li]
    X_test_model_3 = X_test_model[li]
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train_model_3, y_train)
    
    predict_train = logistic_model.predict_proba(X_train_model_3)
    fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
    KS_train = max(tpr-fpr)

    if KS_train>0.27:
        predict_test = logistic_model.predict_proba(X_test_model_3)
        fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
        KS_test = max(tpr-fpr)

        if KS_test>0.23:
            print(X_train_model_3.columns,"KS_train:",KS_train,"KS_test",KS_test)
    
    del li

Index(['af_n_Inquiry_Rejected_180d', 'var44', 'af_n_Query_LoanCheck_all',
       'consumer_orgnization_num', 'apply_admittance_score',
       'jyzx_max_Inquiry_30d', 'af_n_Repay_Clear_90d',
       'jyzx_n_Repay_Unclear_30d', 'af_n_Inquiry_360d',
       'jyzx_n_Inquiry_Pass_360d', 'latest_1mth_query_num',
       'net_loan_orgnization_num', 'jyzx_mean_Period_all', 'var47'],
      dtype='object') KS_train: 0.27584726626983413 KS_test 0.23160722129516703
Index(['var44', 'af_n_Query_LoanCheck_all', 'consumer_orgnization_num',
       'apply_admittance_score', 'jyzx_max_Inquiry_30d',
       'jyzx_n_Inquiry_Rejected_90d', 'af_n_Repay_Clear_90d',
       'latest_6mth_loan_num', 'jyzx_n_Repay_Unclear_30d',
       'jyzx_n_Inquiry_Pass_360d', 'latest_1mth_query_num',
       'net_loan_orgnization_num', 'jyzx_mean_Period_all', 'var47'],
      dtype='object') KS_train: 0.2706461752781697 KS_test 0.2333757771342584
Index(['var44', 'af_n_Query_LoanCheck_all', 'consumer_orgnization_num',
       'apply_ad

#  #

In [95]:
X_train_model = X_train[['af_n_Query_LoanCheck_all', 'af_n_Query_NoLicConFin_all', 'var44',
       'past_with_success_num', 'var47', 'latest_1mth_query_num',
       'af_n_Inquiry_Rejected_30d', 'var55', 'latest_1mth_with_failed_num',
       'af_n_Inquiry_Pass_rate_30d', 'var15', 'af_n_Repay_Clear_all',
       'cridiet_loan_duration', 'loan_rank', 'latest_6mth_query_num',
       'af_sum_Inquiry_Pass_rate_360d', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'
       ]]
X_test_model = X_test[['af_n_Query_LoanCheck_all', 'af_n_Query_NoLicConFin_all', 'var44',
       'past_with_success_num', 'var47', 'latest_1mth_query_num',
       'af_n_Inquiry_Rejected_30d', 'var55', 'latest_1mth_with_failed_num',
       'af_n_Inquiry_Pass_rate_30d', 'var15', 'af_n_Repay_Clear_all',
       'cridiet_loan_duration', 'loan_rank', 'latest_6mth_query_num',
       'af_sum_Inquiry_Pass_rate_360d', 'var16', 'af_n_Inquiry_30d',
       'af_max_Repay_Odue_90d', 'jyzx_n_Inquiry_180d'
       ]]

In [96]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.2978920503290072
KS-test: 0.30133099200895813


In [97]:
predict_train = logistic_model.predict_proba(X_train_model)
prob_pred_train = predict_train[np.argsort(predict_train[:,1])][:,1]
prob_groups = pd.qcut(prob_pred_train, 10)

X_train_model['predict_proba'] = logistic_model.predict_proba(X_train_model)[:,1]
X_train_model['target'] = y_train
X_train_model['prob_groups'] = pd.qcut(X_train_model['predict_proba'], 10)

train = X_train_model[['predict_proba','target','prob_groups']].sort_values('prob_groups')
prob_data_train = pd.DataFrame(index=train['prob_groups'].unique())
prob_data_train['概率下限'] = prob_data_train.index.map(lambda x:x.right)
prob_data_train['概率上限'] = prob_data_train.index.map(lambda x:x.left)
prob_data_train['样本量'] = train['prob_groups'].groupby(prob_groups).count()
prob_data_train['BAD'] = train['target'].groupby(prob_groups).sum()
prob_data_train['GOOD'] = prob_data_train['样本量']-prob_data_train['BAD']
prob_data_train['正例比例'] = (prob_data_train['BAD']/prob_data_train['样本量']).apply(lambda x: format(x, '.2%')) 
prob_data_train

,概率下限,概率上限,样本量,BAD,GOOD,正例比例
"(0.0119, 0.0675]",0.0675,0.0119,572,33,539,5.77%
"(0.0675, 0.0878]",0.0878,0.0675,572,39,533,6.82%
"(0.0878, 0.108]",0.1080,0.0878,571,68,503,11.91%
"(0.108, 0.127]",0.1270,0.1080,572,60,512,10.49%
"(0.127, 0.146]",0.1460,0.1270,571,69,502,12.08%
"(0.146, 0.166]",0.1660,0.1460,572,82,490,14.34%
"(0.166, 0.193]",0.1930,0.1660,571,106,465,18.56%
"(0.193, 0.231]",0.2310,0.1930,572,115,457,20.10%
"(0.231, 0.298]",0.2980,0.2310,571,157,414,27.50%
"(0.298, 0.961]",0.9610,0.2980,572,252,320,44.06%


In [98]:
x_test = X_test_model
predict_test = logistic_model.predict_proba(x_test)[:,1]

x_test['predict_proba'] = logistic_model.predict_proba(x_test)[:,1]
x_test['target'] = y_test


# 将测试集概率值划入训练集分组
bin = [0,1]
for i in prob_groups.unique(): # 训练集分割点
    bin.append(i.left)
bin.sort()
bin.pop(1)
x_test['prob_groups'] = pd.cut(predict_test, bin)
prob_groups_test = pd.cut(predict_test, bin)

test = x_test[['predict_proba','target','prob_groups']].sort_values('prob_groups')

prob_data_test = pd.DataFrame(index=test['prob_groups'].unique())
prob_data_test['概率下限'] = prob_data_test.index.map(lambda x:x.right)
prob_data_test['概率上限'] = prob_data_test.index.map(lambda x:x.left)
prob_data_test['样本量'] = test[['target','prob_groups']].groupby('prob_groups').count()
prob_data_test['BAD'] = test[['target','prob_groups']].groupby('prob_groups').sum()
prob_data_test['GOOD'] = prob_data_test['样本量']-prob_data_test['BAD']
prob_data_test['正例比例'] = (prob_data_test['BAD']/prob_data_test['样本量']).apply(lambda x: format(x, '.2%')) 
prob_data_test

,概率下限,概率上限,样本量,BAD,GOOD,正例比例
"(0.0, 0.0675]",0.0675,0.0000,136,9,127,6.62%
"(0.0675, 0.0878]",0.0878,0.0675,176,6,170,3.41%
"(0.0878, 0.108]",0.1080,0.0878,208,18,190,8.65%
"(0.108, 0.127]",0.1270,0.1080,197,23,174,11.68%
"(0.127, 0.146]",0.1460,0.1270,260,25,235,9.62%
"(0.146, 0.166]",0.1660,0.1460,290,37,253,12.76%
"(0.166, 0.193]",0.1930,0.1660,307,54,253,17.59%
"(0.193, 0.231]",0.2310,0.1930,346,55,291,15.90%
"(0.231, 0.298]",0.2980,0.2310,325,82,243,25.23%
"(0.298, 1.0]",1.0000,0.2980,323,135,188,41.80%


In [99]:
for i in X_test_model.columns:
    print(i)
print(logistic_model.coef_)
print(logistic_model.intercept_)

af_n_Query_LoanCheck_all
af_n_Query_NoLicConFin_all
var44
past_with_success_num
var47
latest_1mth_query_num
af_n_Inquiry_Rejected_30d
var55
latest_1mth_with_failed_num
af_n_Inquiry_Pass_rate_30d
var15
af_n_Repay_Clear_all
cridiet_loan_duration
loan_rank
latest_6mth_query_num
af_sum_Inquiry_Pass_rate_360d
var16
af_n_Inquiry_30d
af_max_Repay_Odue_90d
jyzx_n_Inquiry_180d
predict_proba
target
prob_groups
[[ 5.68553197e-02  2.56427015e-02 -2.90627050e-03 -2.17420414e-02
  -4.73957641e-03  5.94484360e-02  1.48673685e-02  4.53995629e-03
   5.21291670e-02 -7.60976194e-01 -5.27984200e-02 -1.18588554e-02
   5.10513382e-04  6.66490630e-03 -1.11685922e-02 -5.58312638e-02
   1.13023496e-01  2.14314889e-02  4.11667615e-04 -1.30119308e-02]]
[-1.64750532]
